In [1]:
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16

from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint

import warnings
warnings.filterwarnings("ignore")

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sn
import random
from tqdm import tqdm
import os
import cv2
import pickle

from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import load_model
from tensorflow.keras.losses import categorical_crossentropy

import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, roc_auc_score
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Activation, Conv2D, MaxPooling2D,MaxPool2D, BatchNormalization, Input
from tensorflow.keras.optimizers import Adam, Adagrad, Adadelta, RMSprop
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import regularizers

from tensorflow.keras.applications import VGG16, VGG19
from tensorflow.keras.applications.vgg16 import decode_predictions, preprocess_input
from tensorflow.keras.preprocessing import image
#import image

In [2]:
train_dir = 'train/'
val_dir = 'test/'
batch_size = 32
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(48,48),
        batch_size=batch_size,
        color_mode="grayscale",
        class_mode='categorical')

Found 28709 images belonging to 7 classes.


In [3]:
train_datagen = ImageDataGenerator(
     zoom_range = 0.2, 
     shear_range = 0.2, 
     width_shift_range=0.2,
     rotation_range=10,
     height_shift_range=0.2,
     horizontal_flip=True, 
     rescale = 1./255
)

In [4]:
validation_generator = val_datagen.flow_from_directory(
        val_dir,
        target_size=(48,48),
        batch_size=batch_size,
        color_mode="grayscale",
        class_mode='categorical')

Found 7178 images belonging to 7 classes.


In [5]:
model= Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu', input_shape=(48, 48,1)))
model.add(Conv2D(64,(3,3), padding='same', activation='relu' ))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128,(5,5), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
    
model.add(Conv2D(512,(3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(512,(3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten()) 
model.add(Dense(256,activation = 'relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))
    
model.add(Dense(512,activation = 'relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Dense(7, activation='softmax'))

In [7]:
# Compiling the Model
earlyStopping = EarlyStopping(monitor='val_accuracy', min_delta= 0.01, patience=5, verbose=1, restore_best_weights=True)
mcp_save = ModelCheckpoint('vgg16_model.h5', save_best_only=True, monitor='val_accuracy')
red_lr = ReduceLROnPlateau(monitor="val_accuracy", patience=3, verbose=1, factor = 0.1, min_lr=0.001)


model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.0001, decay=1e-6),metrics=['accuracy'])
history = model.fit(
        train_generator,
        steps_per_epoch=28709 // batch_size,
        validation_steps=7178 // batch_size,
        epochs=100,
        validation_data=validation_generator,
        batch_size = 64,
        callbacks=[red_lr,earlyStopping]
)

Epoch 1/100
897/897 [==============================] - 54s 48ms/step - loss: 8.9326 - accuracy: 0.2117 - val_loss: 7.8151 - val_accuracy: 0.2786 - lr: 1.0000e-04
Epoch 2/100
897/897 [==============================] - 36s 40ms/step - loss: 7.1547 - accuracy: 0.2745 - val_loss: 6.2021 - val_accuracy: 0.3073 - lr: 1.0000e-04
Epoch 3/100
897/897 [==============================] - 39s 43ms/step - loss: 5.4784 - accuracy: 0.3350 - val_loss: 4.6348 - val_accuracy: 0.4022 - lr: 1.0000e-04
Epoch 4/100
897/897 [==============================] - 41s 45ms/step - loss: 4.1822 - accuracy: 0.3846 - val_loss: 3.5678 - val_accuracy: 0.4135 - lr: 1.0000e-04
Epoch 5/100
897/897 [==============================] - 39s 43ms/step - loss: 3.2383 - accuracy: 0.4353 - val_loss: 2.8500 - val_accuracy: 0.4534 - lr: 1.0000e-04
Epoch 6/100
897/897 [==============================] - 39s 43ms/step - loss: 2.6191 - accuracy: 0.4759 - val_loss: 2.3927 - val_accuracy: 0.5028 - lr: 1.0000e-04
Epoch 7/100
897/897 [=======

In [29]:
model.save('model.h5')

In [2]:
model = load_model("model.h5")

In [3]:
cv2.ocl.setUseOpenCL(False)
emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}
cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    if not ret:
        break
    bounding_box = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    num_faces = bounding_box.detectMultiScale(gray_frame,scaleFactor=1.3, minNeighbors=5)
    for (x, y, w, h) in num_faces:
        cv2.rectangle(frame, (x, y-50), (x+w, y+h+10), (255, 0, 0), 2)
        roi_gray_frame = gray_frame[y:y + h, x:x + w]
        cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray_frame, (48, 48)), -1), 0)
        emotion_prediction = model.predict(cropped_img)
        maxindex = int(np.argmax(emotion_prediction))
        cv2.putText(frame, emotion_dict[maxindex], (x+20, y-60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
    cv2.imshow('Video', cv2.resize(frame,(1200,860),interpolation = cv2.INTER_CUBIC))
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        break